## 10.5 The algorithm behind Grover’s search
This example, provides some more detail about Grover's algorithm. It shows the quantum circuit for Grover's algorithm, together with the probability vector. This vector gradually moves towards a vector where a single element is getting closer to 100%.

In [ ]:
using ImageShow
using LinearAlgebra
using StrangelyDisplayed
using StrangelyQuantum

Th function below is similar to the code used by `Classic.search` in `StrangelyQuantum`, but with the length of the array and the index of the search item explicitly provided.

In [ ]:
function doGrover(dim, solution)
    sqee = SimpleQuantumExecutionEnvironment()
    nn = 1 << dim
    cnt = π * sqrt(nn) / 4
    p = Program(dim)
    s0 = Step()
    for i = 1:dim
        addGate(s0, Hadamard(i))
    end
    addStep(p, s0)
    oracle = createOracle(dim, solution)
    setCaption(oracle, "O")
    dif = createDiffMatrix(dim)
    difOracle = Oracle(dif)
    setCaption(difOracle, "D")
    i = 1
    while i < cnt
        s0prob = Step("Prob $i")
        addGate(s0prob, ProbabilitiesGate(1))
        s1 = Step("Oracle $i")
        addGate(s1, oracle)
        s1prob = Step("Prob $i")
        addGate(s1prob, ProbabilitiesGate(1))
        s2 = Step("Diffusion $i")
        addGate(s2, difOracle)
        s3 = Step("Prob $i")
        addGate(s3, ProbabilitiesGate(1))
        addStep(p, s0prob)
        addStep(p, s1)
        addStep(p, s1prob)
        addStep(p, s2)
        addStep(p, s3)
        i += 1
    end
    println(" n = ", dim, ", steps = ", cnt)

    res = runProgram(sqee, p)
    i = 1
    while i < cnt
        ip0 = getIntermediateProbability(res, 3 * i + 1)
        println("results after step ", i, ": ", sum(abs2, ip0[solution]))
        i += 1
    end
    println("\n")
    return p
end

The following function constructs the *diffusion matrix* whicis used to amplify the probability of the qubit with the desired index. Some of the theory is described [here](https://www.cs.cmu.edu/~odonnell/quantum15/lecture04.pdf).

In [ ]:
function createDiffMatrix(dim)
    nn = 1 << dim
    g = Hadamard(1)
    h2 = g.matrix
    for i = 2:dim
        h2 = kron(h2, g.matrix)
    end
    I2 = Matrix{ComplexF64}(I, nn, nn)
    I2[1, 1] = -1
    nd = dim << 1

    inter1 = h2 * I2
    dif = inter1 * h2
    return dif
end

The following function creates the oracle for the algorithm:

In [ ]:
function createOracle(dim, solution)
    nn = 1 << dim
    println("dim = ", dim, " hence N = ", nn)
    matrix = Matrix{ComplexF64}(I, nn, nn)
    matrix[solution, solution] = -1
    return Oracle(matrix)
end

A simple case with 2 qubits, matching the third index:

In [ ]:
p_2_3 = doGrover(2, 3)

In [ ]:
drawProgram(p_2_3)

The output index is 1 greater than the binary number represented by the qubits:
$$ 10_b + 1 \rightarrow 0\times 2^0 + 1\times 2^1 + 1 = 3$$

A slightly more realistic example, with 6 qubits (corresponding to data length $64=2^6$) and an index of 11:

In [ ]:
p_6_11 = doGrover(6, 11)

In [ ]:
drawProgram(p_6_11)

Once again, the output index is 1 greater than the binary number represented by the qubits:
$$001010_b + 1 \rightarrow 0\times 2^0 + 1\times 2^1 + 0\times 2^2 + 1\times 2^3 + 0\times 2^4 + 0\times 2^5 + 1 = 11$$